In [21]:
%matplotlib widget

In [22]:
import open3d as o3d
import cv2 as ocv
import os
from matplotlib import pyplot as plt
import numpy as np


# Importing a python kitti toolbox from https://github.com/utiasSTARS/pykitti
import pykitti

In [36]:
currentPath = os.getcwd()
folderPath = (currentPath[:currentPath.find("Code")])
figpath = r"../Writeup/Figures/"

# for (dirpath, dirnames, filenames) in os.walk(folderPath+r"Datasets/"):
#     print(dirnames)

In [24]:
print("Path to the datasets is:")
print(folderPath+r"Datasets/2011_09_26_drive_0001_extract/")

# May help to check if this is the right path on your machine.

Path to the datasets is:
/home/michaelkatsoulis/Documents/UCT/2020 Semester 2/EEE4022S - Final Project/Datasets/2011_09_26_drive_0001_extract/


## About the libraries used:
I am using the pykitti library written by Lee Clement. <br>
The information about the library is avalible at https://github.com/utiasSTARS/pykitti


In [25]:
# basedir = r"../Datasets/KITTI_cvlibs/2011_09_26_drive_0001"
basedir = r"/media/LinData/Datasets/KITTI_cvlibs/2011_09_26_drive_0001"
date = '2011_09_26'
drive = '0001'
data = pykitti.raw(basedir, date, drive, frames=range(0, 50, 5))
# The 'frames' argument is optional - default: None, which loads the whole dataset.
# Calibration, timestamps, and IMU data are read automatically. 
# Camera and velodyne data are available via properties that create generators
# when accessed, or through getter methods that provide random access.

In [26]:
def getPyKittiData():
    import pykitti
    basedir = r"/media/LinData/Datasets/KITTI_cvlibs/2011_09_26_drive_0001"
    date = '2011_09_26'
    drive = '0001'
    data = pykitti.raw(basedir, date, drive, frames=range(0, 50, 5))
    return pykitti.raw(basedir, date, drive, frames=range(0, 50, 5))

""""

In case of error:
  - Check if the data is actually avalible at the correct base directory.
  - There should be a folder with the date in the directory.
  - In that folder (2011_09_26) there should be folders with the calibration, sync, tracklets and data for the drive recorded.
 
"""

'"\n\nIn case of error:\n  - Check if the data is actually avalible at the correct base directory.\n  - There should be a folder with the date in the directory.\n  - In that folder (2011_09_26) there should be folders with the calibration, sync, tracklets and data for the drive recorded.\n \n'

## Time to explore the current configuration data:
 ---


In [27]:
# TODO
# Find out what the different calibration objects represent.


Now to start using the data and trying to display it.
---


In [28]:
im_data = data.get_cam0(0)
# The index used in data.get_cam0(index) is the time index

In [72]:
rgbIms = data.get_rgb(1)




fig_colour, ax1 = plt.subplots(1,1, figsize = (12,4.5))
fig_colour.tight_layout(pad=1)

ax1.imshow(rgbIms[0])
ax1.set_title("Right RGB Camera (Cam2)")

# fig_colour.savefig(figpath+"Cam2TError.png")
# rgbIms = data.get_rgb(5)
fig_colour2, ax2 = plt.subplots(1,1, figsize = (12,4.5))
fig_colour2.tight_layout(pad=1)
ax2.imshow(rgbIms[1])
ax2.set_title("Left RGB Camera (Cam3)")
# fig_colour2.savefig(figpath+"Cam3TError.png")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
velo_pts_1 = data.get_velo(1)


f2 =  plt.figure(figsize=(18,10))
ax2 = f2.add_subplot(111, projection='3d')
# Plot every 100th point so things don't get too bogged down
velo_range = range(0, velo_pts_1.shape[0], 5)
ax2.scatter(velo_pts_1[velo_range, 0],
            velo_pts_1[velo_range, 1],
            velo_pts_1[velo_range, 2],
            c=velo_pts_1[velo_range, 3],
            cmap='gray')
ax2.set_title('Third Velodyne scan (subsampled)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
plt.close("all")

In [91]:
# trying to plot only what's ahead

# vel_infront_1 = velo_pts_1[(velo_pts_1[:,0]>=0) & (velo_pts_1[:,1]>=0),:]
vel_infront_1 = velo_pts_1[(velo_pts_1[:,0]>=0),:]
f3 = plt.figure(num=2, figsize=(15,6))
ax3 = f3.add_subplot(111, projection='3d')
f3.tight_layout(pad=3.0)
# Plot every 100th point so things don't get too bogged down
velo_range2 = range(0, vel_infront_1.shape[0], 5)
ax3.scatter(vel_infront_1[velo_range2, 0],
            vel_infront_1[velo_range2, 1],
            vel_infront_1[velo_range2, 2],
            c=vel_infront_1[velo_range2, 3])#,
#             cmap='gray')
ax3.set_title('Forwards facing Velodyne (subsampled to 1/5 of original density)', fontsize=18)
ax3.set_xlabel('x-axis', fontsize=15)
ax3.set_ylabel('y-axis', fontsize=15)
ax3.set_zlabel('z-axis', fontsize=15)
ax3.view_init(elev=22, azim= -158)
ax3.set_zlim(-3, 10)
# f3.savefig(figpath+"rawForwardsVelodyne.png")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [90]:
plt.close("all")

# Building a GIF of the images when driving

---
Goal: Have a function that can be sent a set of image paths which will be used to make a GIF from the camera's images. 

Note:

Needs ImageMagick as it's function that it's going to call on your machine.

In [15]:
import glob
import os


In [16]:

gif_name = 'outputName'
file_list = glob.glob('*.png') # Get all the pngs in the current directory
list.sort(file_list, key=lambda x: int(x.split('_')[1].split('.png')[0])) # Sort the images by #, this may need to be tweaked for your use case

with open('image_list.txt', 'w') as file:
    for item in file_list:
        file.write("%s\n" % item)

os.system('convert @image_list.txt {}.gif'.format(gif_name))

IndexError: list index out of range